Información a considerar para el siguiente ETL.
- 1 se borrarán las columnas que no influyan en los KPI que se plantearon para este proyecto. 
- 2 debajo de cada conjunto de código de cada datase estará el diccionario explicativo del contenido de cada columna.
 


In [2]:
import pandas as pd

In [2]:
# Configurar para mostrar todas las columnas y filas en la salida
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [3]:
# cargamos cada uno de los dataset
business = pd.read_parquet(r'Dataset_filtrados\business_filtrado_completo')
checkin = pd.read_parquet(r'Dataset_filtrados\Total_checkin')
review = pd.read_parquet(r'Dataset_filtrados\Total_reviews')
tips = pd.read_parquet(r'Dataset_filtrados\Total_tips')
user = pd.read_parquet(r'Dataset_filtrados\Total_user')


# business

In [4]:
#verificamos informacion general
business.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
39,fSCNwMtNNQY9QT69Cj9fiA,Sierra Pro Events,,Sparks,NV,89431,39.540154,-119.748395,5.0,7,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Musicians, DJs, Karaoke, Event Planning & Serv...","{'Friday': '9:0-17:0', 'Monday': '9:0-17:0', '..."


In [5]:
#Visuaisamos columnas
business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

Considerando los kpis, se ha tomado la decisión de eliminar as columnas que no tendrían influencia en estos con el fin de aligerar el peso de los dataset y mejorar su almacenamiento.

Por lo tanto las columnas 'is_open', 'hours', 'attributes’, al no tener información que se utilizara en el sistema de recomendación  y en la medición de los kpis serán eliminadas. 


In [6]:
columnas_a_eliminar = ['is_open', 'hours', 'attributes']
business.drop(columns=columnas_a_eliminar, inplace=True)


In [7]:
#verificamos cambios 
business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'categories'],
      dtype='object')

In [8]:
#verificamos la existencia de datos nulos
business.isna().sum()

business_id     0
name            0
address         0
city            0
state           0
postal_code     0
latitude        0
longitude       0
stars           0
review_count    0
categories      0
dtype: int64

In [9]:
#ratificamos que los estads que se utilizaran son 
# 'NV': Nevada
# 'FL': Florida
# 'CA': California
# 'IL': Illinois
business['state'].unique()

array(['NV', 'FL', 'CA', 'IL'], dtype=object)

In [10]:
business.to_parquet(r'Dataset_filtrados\business_filtrado_completo')

## Diccionario de Bussines 

•	business_id: Un identificador único para cada negocio en la base de datos.

•	name: El nombre del negocio.

•	address: La dirección del negocio.

•	city: La ciudad donde se encuentra el negocio.

•	state: El estado o provincia donde se encuentra el negocio.

•	postal_code: El código postal del área donde se encuentra el negocio.

•	latitude: La latitud geográfica del negocio.

•	longitude: La longitud geográfica del negocio.

•	stars: La calificación promedio del negocio en Yelp, en una escala de 1 a 5 estrellas.

•	review_count: El número total de reseñas que ha recibido el negocio en Yelp.

•	categories: Las categorías o etiquetas que describen el tipo de negocio, separadas por comas.


# checkin

In [11]:
#visuaizamos datos de forma general
checkin.head(1)

,business_id,date
7,--ARBQr1WMsTWiwOKOj-FQ,"2014-12-12 00:44:23, 2015-01-09 00:19:52, 2015..."


In [12]:
#verificamos la existencia de datos nulos
checkin.isna().sum()

business_id    0
date           0
dtype: int64

## Diccionario de checkin 

•	business_id: Es un identificador único para cada negocio en Yelp.

•	date: Representa las fechas en las que se registraron los check-ins para el negocio. Puede haber múltiples fechas separadas por comas para un mismo negocio, lo que sugiere múltiples check-ins en diferentes momentos.


In [41]:
checkin.to_parquet(r'Dataset_filtrados\Total_checkin')


# union de business y checkin

Se ha pensado una nueva forma de abordar el área de negocio, a partir de este dataset, tomando en consideración las visitas a su página web y que hoy en día gran parte del mercadeos se da mediante internet, utilizaremos la columna date  para ver las visitas de un citio y según el promedio de las visitas de los negocios parecidos recomendar que se puede hacer desde el área de marketing

In [13]:
# unimos los dataset de business y checkin para relacional las visitas a cada negocio 
Visitas = pd.merge(business, checkin, on='business_id')

In [14]:
#visuaizamos datos de forma general
Visitas.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,date
0,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.196252,-82.380615,4.5,95,"Burgers, Sports Bars, Bars, Lounges, Restauran...","2018-02-09 22:40:05, 2018-03-25 21:01:21, 2018..."


In [15]:
# A partir de la columna date, que como se menciono anteriormente contiene fechas de las visitas en la pagina web de cada negocio,
# se genera una nueva columna que contiene la cantidad de visitas. 
Visitas['num_Visitas'] = Visitas['date'].apply(lambda x: len(x.split(',')))

In [16]:
#verificamos datos.
Visitas.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,date,num_Visitas
0,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.196252,-82.380615,4.5,95,"Burgers, Sports Bars, Bars, Lounges, Restauran...","2018-02-09 22:40:05, 2018-03-25 21:01:21, 2018...",111


A continuación, se calculará el promedio de visitas de cada nicho o negocio (hoteles, bars, nightlife) y para ello se crea la siguiente función

In [17]:
def asignar_promedio(row):
    #promedio de visitas para hoteles, Aquí se calcula el promedio de visitas de todas los negocios en los cuales la columna categoría contenga la palabra ‘hotel’ 
    promedio_hotel= Visitas[Visitas['categories'].str.lower().str.strip().str.contains('hotel')]['num_Visitas'].mean() 
    #promedio de visitas para nightlife, Aquí se calcula el promedio de visitas de todas los negocios en los cuales la columna categoría contenga la palabra ‘nightlife’ 
    promedio_nightlife = Visitas[Visitas['categories'].str.lower().str.strip().str.contains('nightlife')]['num_Visitas'].mean()
    #promedio de visitas para bars, Aquí se calcula el promedio de visitas de todas los negocios en los cuales la columna categoría contenga la palabra ‘bars’ 
    promedio_bars= Visitas[Visitas['categories'].str.lower().str.strip().str.contains('bars')]['num_Visitas'].mean()
    if 'hotel' in row['categories'].lower():
        return promedio_hotel
    elif 'nightlife' in row['categories'].lower():
        return promedio_nightlife
    elif 'bars' in row['categories'].lower():
        return promedio_bars
    else:
        return None  # O el valor por defecto que desees

In [18]:
#se crea una nueva columna aplicando la función previamente creada
Visitas['promedio_visitas'] = Visitas.apply(asignar_promedio, axis=1)


In [19]:
Visitas.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,date,num_Visitas,promedio_visitas
0,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.196252,-82.380615,4.5,95,"Burgers, Sports Bars, Bars, Lounges, Restauran...","2018-02-09 22:40:05, 2018-03-25 21:01:21, 2018...",111,258.313062


In [20]:
# Función para asignar mensajes en función de si el valor es menor o mayor que el promedio
def resultado(row):
    if row['num_Visitas'] < row['promedio_visitas']:
        return 'menor al promedio'
    elif row['num_Visitas'] > row['promedio_visitas']:
        return 'mayor al promedio'
    else:
        return 'igual al promedio'


In [21]:

# Aplicar la función a cada fila y crear una nueva columna 'mensaje_promedio'
Visitas['mensaje_promedio'] = Visitas.apply(resultado, axis=1)


In [22]:
Visitas.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,date,num_Visitas,promedio_visitas,mensaje_promedio
0,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.196252,-82.380615,4.5,95,"Burgers, Sports Bars, Bars, Lounges, Restauran...","2018-02-09 22:40:05, 2018-03-25 21:01:21, 2018...",111,258.313062,menor al promedio
1,SZU9c8V2GuREDN5KgyHFJw,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,"Live/Raw Food, Restaurants, Seafood, Beer Bar,...","2010-03-07 05:23:49, 2010-03-21 05:07:50, 2010...",6148,258.313062,mayor al promedio
2,7Du9oW73YcYFmXdtU5aiSg,Kinjo's Japanese Restaurant,2875 Tyrone Blvd N,Saint Petersburg,FL,33710,27.797263,-82.733913,3.5,17,"Japanese, Sushi Bars, Restaurants","2010-07-14 22:36:50, 2010-07-15 21:04:21, 2010...",31,253.113694,menor al promedio
3,znL4YrNbA2uwOQ3CchkmEA,Seaview Condominiums,14700 Gulf Blvd,Madeira Beach,FL,33708,27.798503,-82.798750,4.5,8,"Home Services, Vacation Rentals, Real Estate, ...","2015-06-14 01:25:52, 2015-06-17 21:28:10, 2015...",3,152.773946,menor al promedio
4,ZU6NodDOWaabGkeNpOWdXw,530 Pub & Grill,530 Cleveland St,Clearwater,FL,33755,27.965747,-82.798901,4.5,6,"American (Traditional), Restaurants, Burgers, ...",2021-07-30 17:28:16,1,258.313062,menor al promedio


# review

In [4]:
#vemos datos en general
review.head(1)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06


In [5]:
#visualizamos columnas
review.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [6]:
# identificamos datos nulos
review.isna().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

Las columnas a eliminar son ‘review_id’ dado que las claves (ID) de bussines y usser serán suficientes para asociar las review a cada negocio y usuario 

In [7]:
columnas = ['review_id']
review.drop(columns=columnas, inplace=True)

In [8]:
#verificamos resultados
review.head(1)

,user_id,business_id,stars,useful,funny,cool,text,date
0,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06


## Diccionario de review 

•	user_id (cadena): Es el identificador único del usuario que ha escrito la reseña. Cada usuario tiene un id_usuario único que lo distingue de otros usuarios.

•	business_id (cadena): Es el identificador único del negocio que recibió la reseña. Cada negocio tiene una Identificación del negocio única que lo distingue de otros negocios.

•	stars (entero): Es la calificación de la reseña en términos de estrellas. Puede ser un valor entero entre 1 y 5, donde 1 es la calificación más baja y 5 es la calificación más alta.

•	useful (entero): Representa el número de votos útiles que ha recibido la reseña de otros usuarios. Los usuarios pueden votar si una reseña les resultó útil o no.

•	funny (entero): Representa el número de votos de otros usuarios que encontraron la reseña divertida. Los usuarios pueden votar si una reseña les resultó divertida o no.

•	cool (entero): Representa el número de votos de otros usuarios que encontraron la reseña fría o poco útil. Los usuarios pueden votar si una reseña les resultó poco útil.

•	text (cadena): Es el contenido de la reseña escrita por el usuario. Aquí se encuentra el texto completo de la reseña que describe la experiencia del usuario con el negocio.



In [9]:
review.to_parquet(r'Dataset_filtrados\Total_reviews')


# tips

In [28]:
#vemos datos en general
tips.head(1)

,user_id,business_id,text,date,compliment_count
8,VL12EhEdT4OWqGq0nIqkzw,xODBZmX4EmlVvbqtKN7YKg,Tacos,2012-07-27 01:48:24,0


In [29]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66363 entries, 8 to 908908
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           66363 non-null  object        
 1   business_id       66363 non-null  object        
 2   text              66363 non-null  object        
 3   date              66363 non-null  datetime64[ns]
 4   compliment_count  66363 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 3.0+ MB


In [30]:
#visualizamos columnas
tips.columns

Index(['user_id', 'business_id', 'text', 'date', 'compliment_count'], dtype='object')

In [31]:
# identificamos datos nulos
tips.isna().sum()

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64

In [32]:
# observamos dimensiones del dataset
tips.shape

(66363, 5)

Las columna a eliminar es 'date’ ya que en el producto esa fecha no será relevante para la presentación de resultados. 

In [33]:
tips.drop(columns= 'date', inplace = True)

In [34]:
#verificamos resultado
tips.head(10)

,user_id,business_id,text,compliment_count
8,VL12EhEdT4OWqGq0nIqkzw,xODBZmX4EmlVvbqtKN7YKg,Tacos,0
13,MlnuJ7T14CE0JDK2ZIOx5g,MDr7KLYSPkEonvGojNEMBw,Let's go Yankees!,0
15,j2sEA3hiUcwHfq9Ml6M47g,EXYbKA1tocvOK_1tXxZXLA,Don't go for dinner. They close at 6. Really Y...,0
22,phKNHTaokisQkZ7qQbxryA,pWuoHHu0LuVrD0HjadDygg,Crepes please!!!,0
30,jDThlALkraoQLLBYHqY7FQ,9DJhhBqQSu-gTBwaqdhgpQ,Come early for the best service as they fill u...,0
41,XJDLaoN1PerKw2woiKeepA,t0zwddmbOGQOADrAxEPHQQ,Come by for some Jelly Bean sours this weekend!,0
43,nZB33t0RFtq_jTxJJJ6wfA,RNzhLU8zy1L5dBkjRK4xEA,Best steak tartare ever,0
57,9qZVikoJoqfQK9SBSynAGA,l-rGtJt0E7PAklT0IK7oFQ,Great atmosphere... Live music... Excellent dr...,0
69,VWJ8PSz6Sg5_AlBvQyGvpw,PcX8zGIJ_XHTAQAcEOwhLw,"Nice resort, right by the powder white beach!",0
77,2idS8cD2zvsLnV5A9IImGg,ihYtslFvge9B9KK1S_E8rw,Great good and service.,0


Al analizar la columna 'compliment_count', se observa que la gran mayoría de sus valores son 0. Solo hay alrededor de 1079 registros (aproximadamente un 1.6% del total) que contienen valores distintos de 0. Dado que esta proporción es bastante baja, hemos tomado la decisión de eliminar la columna 'compliment_count' de nuestros datos. Esto se debe a que los valores no son lo suficientemente significativos para influir en los resultados que estamos buscando presentar.

In [35]:
#verificamos cantidad de datos distitnso a 0
(tips['compliment_count'] != 0).sum()

1079

In [36]:
#eliinamos la columna 'compliment_count'
tips.drop(columns= 'compliment_count', inplace = True)
#verificamos resultado
tips.head()

,user_id,business_id,text
8,VL12EhEdT4OWqGq0nIqkzw,xODBZmX4EmlVvbqtKN7YKg,Tacos
13,MlnuJ7T14CE0JDK2ZIOx5g,MDr7KLYSPkEonvGojNEMBw,Let's go Yankees!
15,j2sEA3hiUcwHfq9Ml6M47g,EXYbKA1tocvOK_1tXxZXLA,Don't go for dinner. They close at 6. Really Y...
22,phKNHTaokisQkZ7qQbxryA,pWuoHHu0LuVrD0HjadDygg,Crepes please!!!
30,jDThlALkraoQLLBYHqY7FQ,9DJhhBqQSu-gTBwaqdhgpQ,Come early for the best service as they fill u...


## Diccionario de tips 


user_id: Es un identificador único para cada usuario en Yelp que ha dejado un consejo (tip) en un negocio.

business_id: Es un identificador único para el negocio en el que se dejó el consejo.

text: Es el contenido del consejo que el usuario dejó para el negocio. Es un comentario o sugerencia que el usuario comparte.

In [43]:
tips.to_parquet(r'Dataset_filtrados\Total_tips')

# user

In [37]:
#visualisamos datos de manera general
user.head(1)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,3.91,250,65,55,56,18,232,844,467,467,239,180


In [38]:
#observamos columnas
user.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [39]:
# verificamos la existencia de datos nulos.
user.isna().sum()

user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64

Después de un análisis exhaustivo de los datos contenidos en cada columna, hemos llegado a la conclusión de que las siguientes columnas serán eliminadas de nuestro conjunto de datos: 'yelping_since', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer' y 'compliment_photos'. Hemos tomado esta decisión debido a que, en el contexto de mostrar los usuarios con más reseñas en cada negocio o nicho, estas columnas no aportan relevancia significativa. 

In [40]:
#establecemos una variable con las columnas a borrar
columnas_drop= ['yelping_since','elite', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer','compliment_photos']
#borramos columnas.
user.drop(columns= columnas_drop, inplace = True)
#verificamos resultados
user.head()

,user_id,name,review_count,useful,funny,cool,friends,fans,average_stars
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,7217,1259,5994,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,3.91
1,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2086,1010,1003,"LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,3.32
2,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1221,14953,9940,11211,"xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,3.85
3,xoZvMJPDW6Q9pDAXI0e_Ww,Ryan,535,1130,487,573,"6tbXpUIU6upoeqWNDo9k_A, Vlab9b73R5qPLIv6tE4DJA...",31,3.89
4,vVukUtqoLF5BvH_VtQFNoA,Charlene,37,63,3,27,"zkK6c9BcDyqreU0fqI_JLQ, opI1hhhFqElB6pptNH9ZqA...",4,4.51


## Diccionario de user 


•	user_id: Es un identificador único para el usuario. Es una cadena de texto que identifica al usuario en Yelp.

•	name: Es el nombre del usuario. Es una cadena de texto que representa el nombre del usuario en Yelp.

•	review_count: Es la cantidad total de reseñas que el usuario ha escrito en Yelp. Es un número entero que indica cuántas reseñas ha dejado el usuario.


•	useful: Representa la cantidad total de votos "útiles" que el usuario ha recibido en sus reseñas por parte de otros usuarios. Es un número entero que indica cuántos usuarios han encontrado útiles las reseñas del usuario.

•	funny: Representa la cantidad total de votos "graciosos" que el usuario ha recibido en sus reseñas por parte de otros usuarios. Es un número entero que indica cuántos usuarios han encontrado graciosas las reseñas del usuario.

•	cool: Representa la cantidad total de votos "frescos" que el usuario ha recibido en sus reseñas por parte de otros usuarios. Es un número entero que indica cuántos usuarios han encontrado frescas las reseñas del usuario.

•	friends: Indica la cantidad de amigos que el usuario tiene en Yelp. Puede estar en blanco (nulo) si el usuario no ha proporcionado esta información.

•	fans: Número de seguidores que el usuario tiene en Yelp.


In [44]:
user.to_parquet(r'Dataset_filtrados\Total_user')
